# 🖐️ GestureJarvis — Tutorial Completo

## Reconocimiento de Gestos con MediaPipe + PyTorch

Este notebook te guiará paso a paso por:
1. **Recolección de datos** con webcam y MediaPipe
2. **Preprocesamiento** y normalización de landmarks
3. **Entrenamiento** de un modelo MLP en PyTorch
4. **Evaluación** y análisis del modelo

---

## 📦 1. Importar Librerías

Primero importamos todas las dependencias necesarias.

In [ ]:
import os
import sys
import csv
import time
from pathlib import Path

import cv2
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import joblib

import torch
import torch.nn as nn
from torch.utils.data import DataLoader, TensorDataset

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.metrics import classification_report, accuracy_score, confusion_matrix

# Configurar matplotlib
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette("husl")

print("✅ Librerías importadas correctamente")
print(f"PyTorch version: {torch.__version__}")
print(f"CUDA disponible: {torch.cuda.is_available()}")

## 🎯 2. Configuración del Proyecto

Definimos las rutas y constantes principales.

In [ ]:
# Rutas del proyecto
PROJECT_ROOT = Path().absolute()
DATA_DIR = PROJECT_ROOT / "data"
MODELS_DIR = PROJECT_ROOT / "models"
SRC_DIR = PROJECT_ROOT / "src"

# Crear directorios si no existen
DATA_DIR.mkdir(exist_ok=True)
MODELS_DIR.mkdir(exist_ok=True)

# Archivos
CSV_PATH = DATA_DIR / "raw_samples.csv"
MODEL_PATH = MODELS_DIR / "gesture_model.pt"
SCALER_PATH = MODELS_DIR / "scaler.pkl"
ENCODER_PATH = MODELS_DIR / "label_encoder.pkl"

# Añadir src al path para imports
sys.path.insert(0, str(PROJECT_ROOT))

# Constantes
NUM_LANDMARKS = 21
FEATURES_PER_LANDMARK = 3  # x, y, z
TOTAL_FEATURES = NUM_LANDMARKS * FEATURES_PER_LANDMARK  # 63

# Gestos que vamos a reconocer
GESTURES = {
    1: "THUMBS_UP",
    2: "THUMBS_DOWN",
    3: "INDEX_POINT",
    4: "PINCH"
}

print(f"📁 Directorio del proyecto: {PROJECT_ROOT}")
print(f"📊 Dataset: {CSV_PATH}")
print(f"🧠 Modelo: {MODEL_PATH}")
print(f"\n🎯 Gestos a reconocer: {list(GESTURES.values())}")

## 🤚 3. MediaPipe Hands — Detección de Landmarks

MediaPipe detecta **21 landmarks** (puntos clave) en cada mano:

```
     8   12  16  20
     |   |   |   |
  4--7-11-15-19
  |  |   |   |   
  3--6-10-14-18
  |  |   |   |   
  2--5--9-13-17
  |  
  1  
  |  
  0 (muñeca)
```

Cada landmark tiene 3 coordenadas: **(x, y, z)**
- `x, y`: posición normalizada (0-1) en el frame
- `z`: profundidad relativa

**Total: 21 × 3 = 63 features**

In [ ]:
from src.utils import init_mediapipe, detect_hand, extract_landmarks, normalize_landmarks

# Inicializar MediaPipe
landmarker = init_mediapipe(max_num_hands=1, min_detection_confidence=0.7)

print("✅ MediaPipe HandLandmarker inicializado")
print(f"   Detecta hasta 1 mano por frame")
print(f"   Confianza mínima: 70%")

## 📸 4. Recolección de Datos

### Proceso:
1. Capturar frame de la webcam
2. Detectar mano con MediaPipe
3. Extraer 63 features (landmarks)
4. **Normalizar** para robustez
5. Guardar en CSV con etiqueta

### ⚠️ Importante: Normalización

**¿Por qué normalizar?**
- La posición de la mano en el frame varía (izquierda, derecha, arriba, abajo)
- La distancia a la cámara cambia el tamaño
- El modelo debe ser **invariante a posición y escala**

**Método:**
1. **Traslación**: Restar la muñeca (landmark 0) → origen en la muñeca
2. **Escalado**: Dividir por dist(muñeca → middle_mcp) → tamaño normalizado

```python
# Antes: landmarks dependen de posición/tamaño
# Después: solo importa la FORMA del gesto
```

In [ ]:
def collect_samples_interactive():
    """
    Función interactiva para recolectar muestras con la webcam.
    
    Controles:
        1-4: Cambiar gesto
        SPACE: Guardar muestra
        Q: Salir
    """
    from src.utils import draw_landmarks, put_text, get_csv_columns
    
    # Preparar CSV
    csv_exists = CSV_PATH.exists() and CSV_PATH.stat().st_size > 0
    csv_file = open(CSV_PATH, mode="a", newline="", encoding="utf-8")
    writer = csv.writer(csv_file)
    
    if not csv_exists:
        writer.writerow(get_csv_columns())
        csv_file.flush()
    
    # Contar muestras existentes
    sample_count = 0
    if csv_exists:
        with open(CSV_PATH, "r", encoding="utf-8") as f:
            sample_count = max(0, sum(1 for _ in f) - 1)
    
    current_label = "THUMBS_UP"
    
    # Abrir webcam
    cap = cv2.VideoCapture(0)
    if not cap.isOpened():
        print("❌ Error: No se pudo abrir la webcam")
        return
    
    print("="*60)
    print("  🎥 RECOLECCIÓN DE DATOS")
    print("="*60)
    print("  Controles:")
    print("    1 = THUMBS_UP")
    print("    2 = THUMBS_DOWN")
    print("    3 = INDEX_POINT")
    print("    4 = PINCH")
    print("    SPACE = Guardar muestra")
    print("    Q = Salir")
    print(f"  Muestras actuales: {sample_count}")
    print("="*60)
    
    frame_idx = 0
    
    while True:
        ret, frame = cap.read()
        if not ret:
            break
        
        frame = cv2.flip(frame, 1)
        h, w, _ = frame.shape
        rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        
        # Detectar mano
        frame_idx += 1
        timestamp_ms = frame_idx * 33
        result = detect_hand(landmarker, rgb, timestamp_ms)
        
        landmarks_ready = False
        norm_landmarks = None
        
        if result is not None and result.hand_landmarks:
            hand_lms = result.hand_landmarks[0]
            draw_landmarks(frame, hand_lms)
            raw = extract_landmarks(hand_lms)
            norm_landmarks = normalize_landmarks(raw)
            landmarks_ready = True
        
        # Overlay
        put_text(frame, f"Gesto: {current_label}", (10, 30), 0.8, (0, 255, 255))
        put_text(frame, f"Muestras: {sample_count}", (10, 65), 0.7, (255, 255, 255))
        status = "✓ Mano detectada" if landmarks_ready else "✗ Sin mano"
        color = (0, 255, 0) if landmarks_ready else (0, 0, 255)
        put_text(frame, status, (10, 100), 0.7, color)
        put_text(frame, "SPACE=guardar | 1-4=gesto | Q=salir", (10, h - 20), 0.5, (200, 200, 200))
        
        cv2.imshow("GestureJarvis - Recolección", frame)
        
        key = cv2.waitKey(1) & 0xFF
        
        # Cambiar gesto
        if key in [ord('1'), ord('2'), ord('3'), ord('4')]:
            idx = int(chr(key))
            current_label = GESTURES[idx]
            print(f"  🎯 Gesto → {current_label}")
        
        # Guardar
        elif key == ord(' '):
            if landmarks_ready and norm_landmarks is not None:
                row = norm_landmarks.tolist() + [current_label]
                writer.writerow(row)
                csv_file.flush()
                sample_count += 1
                print(f"  ✅ Muestra #{sample_count} guardada — {current_label}")
            else:
                print("  ⚠️  Sin mano detectada")
        
        # Salir
        elif key in [ord('q'), ord('Q')]:
            break
    
    csv_file.close()
    cap.release()
    cv2.destroyAllWindows()
    landmarker.close()
    
    print(f"\n📊 Total de muestras: {sample_count}")
    print(f"💾 Guardadas en: {CSV_PATH}")

# ⚠️ Descomenta la siguiente línea para ejecutar la recolección
# collect_samples_interactive()

### 💡 Recomendaciones para el Dataset

Para un modelo robusto, recolecta:
- **Mínimo**: 200 muestras por gesto (800 total)
- **Ideal**: 300-500 por gesto (1200-2000 total)

**Variabilidad importante:**
- ✅ Diferentes posiciones (centro, izquierda, derecha, arriba, abajo)
- ✅ Diferentes distancias a la cámara
- ✅ Ligeras variaciones en la orientación de la mano
- ✅ Diferentes condiciones de iluminación
- ✅ Ambas manos si quieres soportar las dos

## 📊 5. Análisis Exploratorio del Dataset

Cargamos y exploramos los datos recolectados.

In [ ]:
# Cargar dataset
if not CSV_PATH.exists():
    print("⚠️  Dataset no encontrado. Ejecuta primero collect_samples_interactive()")
else:
    df = pd.read_csv(CSV_PATH)
    
    print(f"📊 Dataset cargado: {len(df)} muestras")
    print(f"📐 Dimensiones: {df.shape}")
    print(f"\n🏷️  Columnas: {df.shape[1]} ({df.shape[1]-1} features + 1 label)")
    
    # Distribución de clases
    print("\n📈 Distribución de gestos:")
    print(df['label'].value_counts())
    
    # Visualizar distribución
    fig, axes = plt.subplots(1, 2, figsize=(14, 5))
    
    # Gráfico de barras
    df['label'].value_counts().plot(kind='bar', ax=axes[0], color='steelblue')
    axes[0].set_title('Distribución de Gestos', fontsize=14, fontweight='bold')
    axes[0].set_xlabel('Gesto')
    axes[0].set_ylabel('Cantidad de Muestras')
    axes[0].grid(axis='y', alpha=0.3)
    
    # Gráfico de pastel
    df['label'].value_counts().plot(kind='pie', ax=axes[1], autopct='%1.1f%%', startangle=90)
    axes[1].set_title('Proporción de Gestos', fontsize=14, fontweight='bold')
    axes[1].set_ylabel('')
    
    plt.tight_layout()
    plt.show()
    
    # Verificar balance
    counts = df['label'].value_counts()
    if counts.max() / counts.min() > 2:
        print("\n⚠️  Dataset desbalanceado. Considera recolectar más muestras del gesto minoritario.")
    else:
        print("\n✅ Dataset bien balanceado")

In [ ]:
# Estadísticas de las features
if CSV_PATH.exists():
    print("📊 Estadísticas de las Features (primeras 10):")
    print(df.iloc[:, :10].describe())
    
    # Visualizar algunas features
    fig, axes = plt.subplots(2, 2, figsize=(14, 10))
    fig.suptitle('Distribución de Landmarks (x0, y0, x1, y1)', fontsize=16, fontweight='bold')
    
    features = ['x0', 'y0', 'x1', 'y1']
    for idx, (ax, feat) in enumerate(zip(axes.flatten(), features)):
        for gesture in df['label'].unique():
            data = df[df['label'] == gesture][feat]
            ax.hist(data, bins=30, alpha=0.6, label=gesture)
        ax.set_title(f'Landmark {feat}', fontweight='bold')
        ax.set_xlabel('Valor')
        ax.set_ylabel('Frecuencia')
        ax.legend()
        ax.grid(alpha=0.3)
    
    plt.tight_layout()
    plt.show()

## 🧠 6. Preparación de Datos para Entrenamiento

### Pipeline de preprocesamiento:
1. Separar features (X) y labels (y)
2. Codificar labels → números (0, 1, 2, 3)
3. Split train/test (80/20)
4. Estandarizar features con **StandardScaler**

In [ ]:
if CSV_PATH.exists():
    # Separar X e y
    X = df.iloc[:, :-1].values.astype(np.float32)  # 63 features
    y = df['label'].values
    
    print(f"📊 X shape: {X.shape}")
    print(f"🏷️  y shape: {y.shape}")
    print(f"\n🎯 Clases únicas: {np.unique(y)}")
    
    # Codificar labels
    le = LabelEncoder()
    y_enc = le.fit_transform(y)
    num_classes = len(le.classes_)
    
    print(f"\n🔢 Label Encoding:")
    for i, label in enumerate(le.classes_):
        print(f"   {label} → {i}")
    
    # Train/Test split
    X_train, X_test, y_train, y_test = train_test_split(
        X, y_enc, 
        test_size=0.2, 
        random_state=42, 
        stratify=y_enc
    )
    
    print(f"\n📂 Split realizado:")
    print(f"   Train: {len(X_train)} muestras ({len(X_train)/len(X)*100:.1f}%)")
    print(f"   Test:  {len(X_test)} muestras ({len(X_test)/len(X)*100:.1f}%)")
    
    # Estandarizar
    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    X_test_scaled = scaler.transform(X_test)
    
    print(f"\n📏 Estandarización:")
    print(f"   Media train: {X_train_scaled.mean():.6f}")
    print(f"   Std train: {X_train_scaled.std():.6f}")
    print(f"   Media test: {X_test_scaled.mean():.6f}")
    print(f"   Std test: {X_test_scaled.std():.6f}")
    
    print("\n✅ Datos preparados para entrenamiento")

## 🏗️ 7. Arquitectura del Modelo

### Multi-Layer Perceptron (MLP)

Red neuronal completamente conectada con 3 capas:

```
Input (63)  →  [Linear(128) → BatchNorm → ReLU → Dropout(0.3)]
            →  [Linear(64)  → BatchNorm → ReLU → Dropout(0.3)]
            →  [Linear(num_classes)]
            →  Softmax (implícito en CrossEntropyLoss)
```

**Componentes:**
- **BatchNorm**: Estabiliza el entrenamiento
- **ReLU**: Función de activación no lineal
- **Dropout**: Regularización (evita overfitting)
- **CrossEntropyLoss**: función de pérdida para clasificación

In [ ]:
class GestureMLP(nn.Module):
    """Red neuronal MLP de 3 capas para clasificación de gestos."""
    
    def __init__(self, input_size=63, num_classes=4):
        super().__init__()
        self.net = nn.Sequential(
            # Capa 1: 63 → 128
            nn.Linear(input_size, 128),
            nn.BatchNorm1d(128),
            nn.ReLU(),
            nn.Dropout(0.3),
            
            # Capa 2: 128 → 64
            nn.Linear(128, 64),
            nn.BatchNorm1d(64),
            nn.ReLU(),
            nn.Dropout(0.3),
            
            # Capa 3: 64 → num_classes
            nn.Linear(64, num_classes),
        )
    
    def forward(self, x):
        return self.net(x)

# Crear modelo
if CSV_PATH.exists():
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    print(f"🖥️  Dispositivo: {device}")
    
    model = GestureMLP(input_size=TOTAL_FEATURES, num_classes=num_classes).to(device)
    
    print(f"\n🧠 Modelo creado:")
    print(model)
    
    # Contar parámetros
    total_params = sum(p.numel() for p in model.parameters())
    trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
    
    print(f"\n📊 Parámetros:")
    print(f"   Total: {total_params:,}")
    print(f"   Entrenables: {trainable_params:,}")

## 🎓 8. Entrenamiento del Modelo

### Hiperparámetros:
- **Epochs**: 50
- **Batch Size**: 32
- **Learning Rate**: 0.001
- **Optimizer**: Adam
- **Loss**: CrossEntropyLoss

In [ ]:
if CSV_PATH.exists():
    # Hiperparámetros
    EPOCHS = 50
    BATCH_SIZE = 32
    LR = 1e-3
    
    # Preparar tensores
    X_train_t = torch.tensor(X_train_scaled, dtype=torch.float32).to(device)
    y_train_t = torch.tensor(y_train, dtype=torch.long).to(device)
    X_test_t = torch.tensor(X_test_scaled, dtype=torch.float32).to(device)
    y_test_t = torch.tensor(y_test, dtype=torch.long).to(device)
    
    # DataLoader
    train_ds = TensorDataset(X_train_t, y_train_t)
    train_loader = DataLoader(train_ds, batch_size=BATCH_SIZE, shuffle=True)
    
    # Loss y Optimizer
    criterion = nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=LR)
    
    print("🚀 Iniciando entrenamiento...\n")
    print("="*60)
    
    # Listas para tracking
    train_losses = []
    train_accs = []
    
    # Loop de entrenamiento
    for epoch in range(1, EPOCHS + 1):
        model.train()
        epoch_loss = 0.0
        correct = 0
        total = 0
        
        for batch_X, batch_y in train_loader:
            # Forward
            optimizer.zero_grad()
            outputs = model(batch_X)
            loss = criterion(outputs, batch_y)
            
            # Backward
            loss.backward()
            optimizer.step()
            
            # Métricas
            epoch_loss += loss.item() * batch_X.size(0)
            preds = outputs.argmax(dim=1)
            correct += (preds == batch_y).sum().item()
            total += batch_X.size(0)
        
        avg_loss = epoch_loss / total
        train_acc = correct / total
        
        train_losses.append(avg_loss)
        train_accs.append(train_acc)
        
        if epoch % 5 == 0 or epoch == 1:
            print(f"Epoch {epoch:3d}/{EPOCHS} | Loss: {avg_loss:.4f} | Acc: {train_acc:.4f}")
    
    print("="*60)
    print("\n✅ Entrenamiento completado")

## 📈 9. Visualización del Entrenamiento

In [ ]:
if CSV_PATH.exists():
    fig, axes = plt.subplots(1, 2, figsize=(14, 5))
    
    # Loss
    axes[0].plot(range(1, EPOCHS + 1), train_losses, linewidth=2, color='crimson')
    axes[0].set_title('Evolución de la Pérdida (Loss)', fontsize=14, fontweight='bold')
    axes[0].set_xlabel('Epoch')
    axes[0].set_ylabel('Loss')
    axes[0].grid(alpha=0.3)
    
    # Accuracy
    axes[1].plot(range(1, EPOCHS + 1), train_accs, linewidth=2, color='forestgreen')
    axes[1].set_title('Evolución de la Precisión (Accuracy)', fontsize=14, fontweight='bold')
    axes[1].set_xlabel('Epoch')
    axes[1].set_ylabel('Accuracy')
    axes[1].set_ylim([0, 1])
    axes[1].grid(alpha=0.3)
    
    plt.tight_layout()
    plt.show()
    
    print(f"📊 Loss final: {train_losses[-1]:.4f}")
    print(f"📊 Accuracy final: {train_accs[-1]:.4f}")

## 🎯 10. Evaluación en Test Set

Ahora evaluamos el modelo en datos que **nunca ha visto** durante el entrenamiento.

In [ ]:
if CSV_PATH.exists():
    model.eval()
    
    with torch.no_grad():
        test_outputs = model(X_test_t)
        test_preds = test_outputs.argmax(dim=1).cpu().numpy()
    
    y_test_np = y_test_t.cpu().numpy()
    
    # Accuracy
    test_acc = accuracy_score(y_test_np, test_preds)
    
    print(f"🎯 Test Accuracy: {test_acc:.4f} ({test_acc*100:.2f}%)")
    print("\n" + "="*60)
    print("📊 Classification Report:")
    print("="*60)
    print(classification_report(
        y_test_np, 
        test_preds, 
        target_names=le.classes_,
        zero_division=0
    ))

## 🔥 11. Matriz de Confusión

Muestra qué gestos se confunden entre sí.

In [ ]:
if CSV_PATH.exists():
    cm = confusion_matrix(y_test_np, test_preds)
    
    plt.figure(figsize=(10, 8))
    sns.heatmap(
        cm, 
        annot=True, 
        fmt='d', 
        cmap='Blues',
        xticklabels=le.classes_,
        yticklabels=le.classes_,
        cbar_kws={'label': 'Cantidad'}
    )
    plt.title('Matriz de Confusión', fontsize=16, fontweight='bold', pad=20)
    plt.xlabel('Predicción', fontsize=12)
    plt.ylabel('Real', fontsize=12)
    plt.tight_layout()
    plt.show()
    
    print("💡 Interpretación:")
    print("   - Diagonal: Predicciones correctas")
    print("   - Fuera de diagonal: Confusiones entre gestos")

## 💾 12. Guardar Modelo y Artefactos

Guardamos:
1. **Modelo** (weights + arquitectura)
2. **Scaler** (para normalizar nuevos datos)
3. **Label Encoder** (para decodificar predicciones)

In [ ]:
if CSV_PATH.exists():
    # Guardar modelo
    torch.save({
        'model_state_dict': model.state_dict(),
        'input_size': TOTAL_FEATURES,
        'num_classes': num_classes,
        'classes': list(le.classes_),
        'test_accuracy': test_acc,
    }, MODEL_PATH)
    
    # Guardar scaler
    joblib.dump(scaler, SCALER_PATH)
    
    # Guardar encoder
    joblib.dump(le, ENCODER_PATH)
    
    print("💾 Artefactos guardados:")
    print(f"   ✅ Modelo: {MODEL_PATH}")
    print(f"   ✅ Scaler: {SCALER_PATH}")
    print(f"   ✅ LabelEncoder: {ENCODER_PATH}")
    print(f"\n🎉 ¡Todo listo para usar en producción!")
    print(f"\n▶️  Ejecuta: python src/realtime_demo.py")

## 🚀 13. Resumen y Próximos Pasos

### ✅ Lo que hemos logrado:
1. ✅ Recolección de datos con MediaPipe
2. ✅ Normalización de landmarks para robustez
3. ✅ Entrenamiento de un MLP en PyTorch
4. ✅ Evaluación exhaustiva del modelo
5. ✅ Guardado de artefactos para producción

### 📈 Mejoras posibles:
- 🔸 Aumentar el dataset (>500 muestras/gesto)
- 🔸 Data augmentation (rotaciones, ruido, etc.)
- 🔸 Probar arquitecturas más profundas
- 🔸 Agregar más gestos
- 🔸 Fine-tuning de hiperparámetros
- 🔸 Usar secuencias temporales (LSTM/GRU)

### 🎮 Demo en tiempo real:
```bash
python src/realtime_demo.py
```

**Gestos:**
- 👍 THUMBS_UP → Subir volumen
- 👎 THUMBS_DOWN → Bajar volumen
- ☝️ INDEX_POINT → Modo ratón
- 🤏 PINCH → Click izquierdo

---

## 🎓 Conceptos Clave Aprendidos

1. **Computer Vision**: Detección de landmarks con MediaPipe
2. **Feature Engineering**: Normalización para invarianza
3. **Deep Learning**: Arquitectura MLP con PyTorch
4. **ML Pipeline**: Train/test split, scaling, encoding
5. **Evaluación**: Accuracy, precision, recall, F1-score
6. **Producción**: Serialización de modelos y artefactos

---

**🙌 ¡Felicidades! Has construido un sistema completo de reconocimiento de gestos.**